# 重庆大学操作系统实验三
## 线程的调度：优先级调度
### 步骤一：
在"struct tcb"中增加线程的静态优先级nice<br>
一定要加在kstack字段后  signature字段前<br>


In [ ]:
//文件位置 /kernel/kernel.h

struct tcb {
    /*hardcoded*/
    uint32_t    kstack;      /*saved top of the kernel stack for this task*/

    int         tid;         /* task id */
    int         state;       /* -1:waiting,0:running,1:ready,2:zombie */
#define TASK_STATE_WAITING  -1
#define TASK_STATE_READY     1
#define TASK_STATE_ZOMBIE    2

    int         timeslice;   //时间片
#define TASK_TIMESLICE_DEFAULT 4

    int         code_exit;   //保存该线程的退出代码
    struct wait_queue *wq_exit; //等待该线程退出的队列

    struct tcb  *next;
    struct fpu   fpu;        //数学协处理器的寄存器
#define NZERO 20
    int nice;                //实验3/step1
    fixedpt estcpu;
    fixedpt priority; //实验3/1
    
    uint32_t     signature;  //必须是最后一个字段
#define TASK_SIGNATURE 0x20160201
};

在函数sys_task_create中初始化nice=0<br>
nice 是整数，取值范围[-NZERO，NZERO-1]，值越小优先级越高

In [ ]:
//文件位置 /kernel/task.c

struct tcb *sys_task_create(void *tos,
                            void (*func)(void *pv), void *pv)
{
    static int tid = 0;
    struct tcb *new;
    char *p;
    uint32_t flags;
    uint32_t ustack=(uint32_t)tos;

    if(ustack & 3)
        return NULL;

    p = (char *)kmemalign(PAGE_SIZE, PAGE_SIZE);
    if(p == NULL)
        return NULL;

    new = (struct tcb *)p;

    memset(new, 0, sizeof(struct tcb));

    new->kstack = (uint32_t)(p+PAGE_SIZE);
    new->tid = tid++;
    new->state = TASK_STATE_READY;
    new->timeslice = TASK_TIMESLICE_DEFAULT;
    new->wq_exit = NULL;
    new->next = NULL;
    new->estcpu = 0;
    new->nice = 0;         //实验3/1
    new->signature = TASK_SIGNATURE;

    /*XXX - should be elsewhere*/
    new->fpu.cwd = 0x37f;
    new->fpu.twd = 0xffff;

    INIT_TASK_CONTEXT(ustack, new->kstack, func, pv);

    save_flags_cli(flags);
    add_task(new);
    restore_flags(flags);

    return new;
}

### 步骤二
增加系统调用
int priotiry(int tid)<br>
    成功返回线程tid的（nice+NZERO）,失败返回-1<br>
int setpriority（int tid,int prio）<br>
    把线程的tid的nice设为(prio-NZERO)<br>
    prio必须在[0，2*NZERO-1]<br>
    成功返回0，失败返回-1



### 步骤三
在struct tcb中再增加两个属性<br>
estcpu：表示线程最近使用了多少cpu时间<br>
<br>待填充<br>
priority：表示线程的动态优先级


### 步骤四：增加一个全局变量
g_load_avg：表示系统的平均负荷<br>
初值为0<br>
每秒更新一次<br>
这里公式


### 步骤五：计算属性

### 步骤六：修改schedule函数，实现优先级调度算法